In [1]:
import numpy as np
import pandas as pd
import datetime as dt

from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer

import matplotlib.pyplot as plt
import plotly.graph_objects as go

# allow more data columns/rows to be shown than by default
pd.set_option('display.max_columns', 501)
pd.set_option('display.max_rows', 501)

In [2]:
# historical FPL data from https://github.com/vaastav/Fantasy-Premier-League
fpl_data = pd.read_csv('../../../repos/Fantasy-Premier-League/data/cleaned_merged_seasons.csv', index_col=0,
                      dtype={'team_x':str})
fpl_data

,season_x,name,position,team_x,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,opp_team_name,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,2016-17,Aaron Cresswell,DEF,NaN,0,0,0,0,0.0,454,10,0,0,0.0,0.0,2016-08-15T19:00:00Z,0,4,Chelsea,0,0,0,0,1,0,14023,1.0,2.0,0.0,0,0,0,0,55,False,0,1
1,2016-17,Aaron Lennon,MID,NaN,0,0,6,0,0.3,142,3,0,0,0.9,8.2,2016-08-13T14:00:00Z,15,17,Spurs,0,0,0,0,1,0,13918,1.0,1.0,0.0,1,0,0,0,60,True,0,1
2,2016-17,Aaron Ramsey,MID,NaN,0,0,5,0,4.9,16,8,3,0,3.0,2.2,2016-08-14T15:00:00Z,60,9,Liverpool,0,0,0,0,1,0,163170,4.0,3.0,23.0,2,0,0,0,80,True,0,1
3,2016-17,Abdoulaye Doucouré,MID,NaN,0,0,0,0,0.0,482,7,0,0,0.0,0.0,2016-08-13T14:00:00Z,0,13,Southampton,0,0,0,0,1,0,1051,1.0,1.0,0.0,0,0,0,0,50,False,0,1
4,2016-17,Adam Forshaw,MID,NaN,0,0,3,0,1.3,286,6,1,0,0.3,2.0,2016-08-13T14:00:00Z,69,14,Stoke,0,0,0,0,1,0,2723,1.0,1.0,0.0,1,0,0,0,45,True,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98397,2021-22,Wilfred Ndidi,MID,Leicester,0,0,0,0,0.0,216,377,0,0,0.0,0.0,2022-05-22T15:00:00Z,0,16,Southampton,0,0,0,0,38,0,70366,1.0,4.0,0.0,0,-202,22,224,48,True,0,38
98398,2021-22,Matt Ritchie,DEF,Newcastle,0,0,3,0,0.0,292,374,0,0,0.0,0.0,2022-05-22T15:00:00Z,1,5,Burnley,0,0,0,0,38,0,15999,2.0,1.0,0.0,1,143,396,253,49,False,0,38
98399,2021-22,Nathan Redmond,MID,Southampton,0,0,5,0,0.0,336,377,3,0,0.0,0.0,2022-05-22T15:00:00Z,82,9,Leicester,0,0,0,0,38,0,13856,1.0,4.0,0.0,2,455,683,228,59,False,0,38
98400,2021-22,Mathew Ryan,GK,Brighton,0,0,0,0,0.0,65,373,0,0,0.0,0.0,2022-05-22T15:00:00Z,0,19,West Ham,0,0,0,0,38,0,587,1.0,3.0,0.0,0,-2,0,2,45,True,0,38


In [3]:
# drop 2016-17 season as fbref data starts from 2017-28
fpl_data = fpl_data[fpl_data.season_x!='2016-17']
fpl_data

,season_x,name,position,team_x,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,opp_team_name,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
8117,2017-18,Aaron Cresswell,DEF,NaN,0,0,3,0,0.6,443,9,2,0,1.9,0.4,2017-08-13T15:00:00Z,9,12,Man Utd,0,0,0,0,1,0,25136,0.0,4.0,18.0,0,0,0,0,50,False,0,1
8118,2017-18,Aaron Lennon,MID,NaN,0,0,0,0,0.0,153,5,0,0,0.0,0.0,2017-08-12T14:00:00Z,0,15,Stoke,0,0,0,0,1,0,4681,0.0,1.0,0.0,0,0,0,0,55,True,0,1
8119,2017-18,Aaron Mooy,MID,NaN,1,0,22,1,46.9,172,4,0,0,8.7,40.2,2017-08-12T14:00:00Z,90,6,Crystal Palace,0,0,0,0,1,0,59955,3.0,0.0,0.0,6,0,0,0,55,False,0,1
8120,2017-18,Aaron Ramsey,MID,NaN,0,0,16,0,11.2,18,1,0,1,6.7,29.6,2017-08-11T18:45:00Z,23,9,Leicester,0,0,0,0,1,0,33792,3.0,4.0,26.0,6,0,0,0,70,True,0,1
8121,2017-18,Abdoulaye Doucouré,MID,NaN,0,2,36,0,25.2,414,7,3,1,10.9,48.6,2017-08-12T11:30:00Z,90,10,Liverpool,0,0,0,0,1,0,1207,3.0,3.0,35.0,9,0,0,0,50,True,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98397,2021-22,Wilfred Ndidi,MID,Leicester,0,0,0,0,0.0,216,377,0,0,0.0,0.0,2022-05-22T15:00:00Z,0,16,Southampton,0,0,0,0,38,0,70366,1.0,4.0,0.0,0,-202,22,224,48,True,0,38
98398,2021-22,Matt Ritchie,DEF,Newcastle,0,0,3,0,0.0,292,374,0,0,0.0,0.0,2022-05-22T15:00:00Z,1,5,Burnley,0,0,0,0,38,0,15999,2.0,1.0,0.0,1,143,396,253,49,False,0,38
98399,2021-22,Nathan Redmond,MID,Southampton,0,0,5,0,0.0,336,377,3,0,0.0,0.0,2022-05-22T15:00:00Z,82,9,Leicester,0,0,0,0,38,0,13856,1.0,4.0,0.0,2,455,683,228,59,False,0,38
98400,2021-22,Mathew Ryan,GK,Brighton,0,0,0,0,0.0,65,373,0,0,0.0,0.0,2022-05-22T15:00:00Z,0,19,West Ham,0,0,0,0,38,0,587,1.0,3.0,0.0,0,-2,0,2,45,True,0,38


In [4]:
fpl17_18 = fpl_data[fpl_data.season_x=='2017-18'].copy()
fpl17_18['kickoff_time'] = pd.to_datetime(fpl17_18['kickoff_time'], yearfirst=True)
fpl17_18

,season_x,name,position,team_x,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,opp_team_name,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
8117,2017-18,Aaron Cresswell,DEF,NaN,0,0,3,0,0.6,443,9,2,0,1.9,0.4,2017-08-13 15:00:00+00:00,9,12,Man Utd,0,0,0,0,1,0,25136,0.0,4.0,18.0,0,0,0,0,50,False,0,1
8118,2017-18,Aaron Lennon,MID,NaN,0,0,0,0,0.0,153,5,0,0,0.0,0.0,2017-08-12 14:00:00+00:00,0,15,Stoke,0,0,0,0,1,0,4681,0.0,1.0,0.0,0,0,0,0,55,True,0,1
8119,2017-18,Aaron Mooy,MID,NaN,1,0,22,1,46.9,172,4,0,0,8.7,40.2,2017-08-12 14:00:00+00:00,90,6,Crystal Palace,0,0,0,0,1,0,59955,3.0,0.0,0.0,6,0,0,0,55,False,0,1
8120,2017-18,Aaron Ramsey,MID,NaN,0,0,16,0,11.2,18,1,0,1,6.7,29.6,2017-08-11 18:45:00+00:00,23,9,Leicester,0,0,0,0,1,0,33792,3.0,4.0,26.0,6,0,0,0,70,True,0,1
8121,2017-18,Abdoulaye Doucouré,MID,NaN,0,2,36,0,25.2,414,7,3,1,10.9,48.6,2017-08-12 11:30:00+00:00,90,10,Liverpool,0,0,0,0,1,0,1207,3.0,3.0,35.0,9,0,0,0,50,True,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18794,2017-18,Willian Borges Da Silva,MID,NaN,1,0,15,0,1.5,106,81,0,0,0.9,3.0,2017-10-21 11:30:00+00:00,22,18,Watford,0,0,0,0,9,0,471910,2.0,4.0,4.0,4,-48410,6822,55232,69,True,0,9
18795,2017-18,Willy Caballero,GK,NaN,0,0,0,0,0.0,96,81,0,0,0.0,0.0,2017-10-21 11:30:00+00:00,0,18,Watford,0,0,0,0,9,0,5979,2.0,4.0,0.0,0,-105,34,139,49,True,0,9
18796,2017-18,Will Hughes,MID,NaN,0,0,0,0,0.0,416,81,0,0,0.0,0.0,2017-10-21 11:30:00+00:00,0,5,Chelsea,0,0,0,0,9,0,6102,2.0,4.0,0.0,0,-201,144,345,47,False,0,9
18797,2017-18,Winston Reid,DEF,NaN,0,0,15,0,1.0,442,90,3,0,3.9,12.0,2017-10-20 19:00:00+00:00,90,3,Brighton,0,0,0,0,9,0,50541,3.0,0.0,26.0,1,423,1416,993,49,True,0,9


In [5]:
fpl17_18.isnull().sum(axis=0)

season_x                 0
name                     0
position                 0
team_x               10682
assists                  0
bonus                    0
bps                      0
clean_sheets             0
creativity               0
element                  0
fixture                  0
goals_conceded           0
goals_scored             0
ict_index                0
influence                0
kickoff_time             0
minutes                  0
opponent_team            0
opp_team_name            0
own_goals                0
penalties_missed         0
penalties_saved          0
red_cards                0
round                    0
saves                    0
selected                 0
team_a_score             0
team_h_score             0
threat                   0
total_points             0
transfers_balance        0
transfers_in             0
transfers_out            0
value                    0
was_home                 0
yellow_cards             0
GW                       0
d

In [6]:
fbref17_18 = pd.read_csv('../../../football-data-analysis/data/player_data/from_fbref_fixtures/premier-league-17-18.csv',
                        index_col=0, header=[0,1])
fbref17_18

General                                     Performance  \
                    Player     #   Nation    Pos     Age   Min         Gls   
0      Alexandre Lacazette   9.0   fr FRA  FW,LW  26-075  90.0           1   
1            Danny Welbeck  23.0  eng ENG  AM,RW  26-258  74.0           1   
2             Theo Walcott  14.0  eng ENG     RW  28-148  16.0           0   
3               Mesut Özil  11.0   de GER     AM  28-300  90.0           0   
4             Granit Xhaka  29.0   ch SUI     DM  24-318  90.0           0   
...                    ...   ...      ...    ...     ...   ...         ...   
10445        Harry Maguire  15.0  eng ENG     CB  25-069  90.0           0   
10446           Wes Morgan   5.0   jm JAM     CB  34-112  90.0           0   
10447        Danny Simpson   2.0  eng ENG     RB  31-129  55.0           0   
10448      Hamza Choudhury  38.0  eng ENG     RB  20-224  35.0           0   
10449      Eldin Jakupović  17.0   ch SUI     GK  33-223  90.0           0   

                                                                  Expected  \
      Ast PK PKatt Sh SoT CrdY CrdR Touches Press  Tkl Int Blocks       xG   
0       0  0     0  3   2    0    0    36.0  18.0  1.0   0    1.0      0.3   
1       0  0     0  3   1    0    0    32.0  10.0  1.0   3    1.0      0.9   
2       0  0     0  0   0    0    0     3.0   1.0  0.0   1    0.0      0.0   
3       0  0     0  3   0    0    0    96.0   8.0  2.0   0    0.0      0.2   
4       2  0     0  4   1    0    0   115.0   8.0  1.0   2    1.0      0.1   
...    .. ..   ... ..  ..  ...  ...     ...   ...  ...  ..    ...      ...   
10445   0  0     0  4   1    0    0    45.0  11.0  2.0   4    2.0      0.3   
10446   0  0     0  0   0    0    0    30.0   7.0  2.0   4    2.0      0.0   
10447   0  0     0  0   0    0    0    24.0   8.0  0.0   2    2.0      0.0   
10448   0  0     0  1   0    1    0    16.0   8.0  1.0   5    0.0      0.0   
10449   0  0     0  0   0    0    0    30.0   0.0  0.0   0    0.0      0.0   

                  SCA      Passes                     Carries       Dribbles  \
      npxG   xA   SCA  GCA    Cmp    Att   Cmp%  Prog Carries  Prog     Succ   
0      0.3  0.3   7.0  2.0   21.0   25.0   84.0   1.0    18.0   1.0      1.0   
1      0.9  0.3   4.0  0.0   16.0   20.0   80.0   1.0    22.0   6.0      2.0   
2      0.0  0.0   1.0  0.0    1.0    2.0   50.0   0.0     2.0   1.0      0.0   
3      0.2  0.3   9.0  0.0   70.0   87.0   80.5   4.0    70.0   7.0      1.0   
4      0.1  0.3  12.0  2.0   86.0  104.0   82.7  15.0    71.0  10.0      1.0   
...    ...  ...   ...  ...    ...    ...    ...   ...     ...   ...      ...   
10445  0.3  0.0   1.0  0.0   20.0   28.0   71.4   3.0    17.0   1.0      1.0   
10446  0.0  0.0   0.0  0.0   17.0   19.0   89.5   1.0     4.0   0.0      0.0   
10447  0.0  0.0   0.0  0.0    6.0   16.0   37.5   0.0     7.0   0.0      0.0   
10448  0.0  0.0   0.0  0.0    9.0    9.0  100.0   1.0    10.0   1.0      0.0   
10449  0.0  0.0   2.0  1.0   18.0   26.0   69.2   0.0     9.0   0.0      0.0   

                   General                          Total                \
       Att           Squad HomeTeam        Opponent   Cmp    Att   Cmp%   
0      1.0         Arsenal     True  Leicester City  21.0   25.0   84.0   
1      2.0         Arsenal     True  Leicester City  16.0   20.0   80.0   
2      0.0         Arsenal     True  Leicester City   1.0    2.0   50.0   
3      3.0         Arsenal     True  Leicester City  70.0   87.0   80.5   
4      1.0         Arsenal     True  Leicester City  86.0  104.0   82.7   
...    ...             ...      ...             ...   ...    ...    ...   
10445  3.0  Leicester City    False       Tottenham  20.0   28.0   71.4   
10446  0.0  Leicester City    False       Tottenham  17.0   19.0   89.5   
10447  0.0  Leicester City    False       Tottenham   6.0   16.0   37.5   
10448  2.0  Leicester City    False       Tottenham   9.0    9.0  100.0   
10449  0.0  Leicester City    False       Totten

In [7]:
fpl17_18[fpl17_18['name']=='Aaron Cresswell']

,season_x,name,position,team_x,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,opp_team_name,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
8117,2017-18,Aaron Cresswell,DEF,NaN,0,0,3,0,0.6,443,9,2,0,1.9,0.4,2017-08-13 15:00:00+00:00,9,12,Man Utd,0,0,0,0,1,0,25136,0.0,4.0,18.0,0,0,0,0,50,False,0,1
8372,2017-18,Aaron Cresswell,DEF,NaN,1,0,15,0,11.2,443,95,0,0,3.7,23.6,2017-10-28 14:00:00+00:00,45,6,Crystal Palace,0,0,0,0,10,0,61476,2.0,2.0,2.0,4,-9169,1213,10382,50,False,0,10
8644,2017-18,Aaron Cresswell,DEF,NaN,0,0,8,0,1.8,443,109,4,0,1.4,12.6,2017-11-04 17:30:00+00:00,90,10,Liverpool,0,0,0,0,11,0,53794,4.0,1.0,0.0,0,-7903,965,8868,50,True,0,11
8917,2017-18,Aaron Cresswell,DEF,NaN,0,0,10,0,1.4,443,119,2,0,0.8,4.6,2017-11-19 16:00:00+00:00,90,18,Watford,0,0,0,0,12,0,50796,0.0,2.0,2.0,1,-3265,1484,4749,50,False,0,12
9191,2017-18,Aaron Cresswell,DEF,NaN,0,0,14,0,16.2,443,129,1,0,3.6,20.2,2017-11-24 20:00:00+00:00,90,9,Leicester,0,0,0,0,13,0,48510,1.0,1.0,0.0,2,-2472,1160,3632,50,True,0,13
9465,2017-18,Aaron Cresswell,DEF,NaN,0,0,8,0,1.9,443,138,4,0,1.2,4.4,2017-11-29 20:00:00+00:00,90,7,Everton,0,0,0,0,14,0,45409,0.0,4.0,6.0,0,-3202,647,3849,50,False,0,14
9740,2017-18,Aaron Cresswell,DEF,NaN,1,1,27,0,28.3,443,147,2,0,7.8,49.2,2017-12-03 16:00:00+00:00,90,11,Man City,0,0,0,0,15,0,40264,1.0,2.0,0.0,5,-5216,234,5450,50,False,0,15
10017,2017-18,Aaron Cresswell,DEF,NaN,0,0,22,1,11.3,443,159,0,0,3.0,18.6,2017-12-09 12:30:00+00:00,90,5,Chelsea,0,0,0,0,16,0,39148,0.0,1.0,0.0,5,-1314,819,2133,50,True,1,16
10294,2017-18,Aaron Cresswell,DEF,NaN,0,0,27,1,1.1,443,170,0,0,2.3,21.6,2017-12-13 20:00:00+00:00,90,1,Arsenal,0,0,0,0,17,0,41325,0.0,0.0,0.0,6,1933,2931,998,50,True,0,17
10572,2017-18,Aaron Cresswell,DEF,NaN,0,2,30,1,15.4,443,178,0,0,4.5,29.6,2017-12-16 16:00:00+00:00,90,15,Stoke,0,0,0,0,18,0,49941,3.0,0.0,0.0,8,8227,9230,1003,50,False,0,18


In [8]:
fbref17_18[fbref17_18[('General','Player')]=='Aaron Cresswell']

General                                    Performance         \
                Player    #   Nation    Pos     Age   Min         Gls Ast PK   
272    Aaron Cresswell  3.0  eng ENG     LB  27-241  10.0           0   0  0   
356    Aaron Cresswell  3.0  eng ENG     LB  27-247  90.0           0   0  0   
686    Aaron Cresswell  3.0  eng ENG     LB  27-254  90.0           0   0  0   
1086   Aaron Cresswell  3.0  eng ENG     WB  27-270  90.0           0   0  0   
1263   Aaron Cresswell  3.0  eng ENG     WB  27-275  90.0           0   0  0   
1388   Aaron Cresswell  3.0  eng ENG  WB,LB  27-282  90.0           0   0  0   
1721   Aaron Cresswell  3.0  eng ENG     LB  27-289  90.0           0   0  0   
2012   Aaron Cresswell  3.0  eng ENG     LB  27-303  90.0           0   0  0   
2559   Aaron Cresswell  3.0  eng ENG     WB  27-317  45.0           0   1  0   
2902   Aaron Cresswell  3.0  eng ENG  WB,LB  27-324  90.0           0   0  0   
3275   Aaron Cresswell  3.0  eng ENG     LB  27-339  90.0           0   0  0   
3314   Aaron Cresswell  3.0  eng ENG     LB  27-344  90.0           0   0  0   
3819   Aaron Cresswell  3.0  eng ENG     LB  27-349  90.0           0   0  0   
4123   Aaron Cresswell  3.0  eng ENG     CB  27-353  90.0           0   1  0   
4136   Aaron Cresswell  3.0  eng ENG     CB  27-359  90.0           0   0  0   
4607   Aaron Cresswell  3.0  eng ENG     CB  27-363  90.0           0   0  0   
4842   Aaron Cresswell  3.0  eng ENG     CB  28-001  90.0           0   0  0   
5161   Aaron Cresswell  3.0  eng ENG     CB  28-008  90.0           0   0  0   
5286   Aaron Cresswell  3.0  eng ENG     CB  28-011  90.0           0   1  0   
5904   Aaron Cresswell  3.0  eng ENG  CB,LB  28-018  75.0           0   1  0   
6161   Aaron Cresswell  3.0  eng ENG     CB  28-029  90.0           0   0  0   
6477   Aaron Cresswell  3.0  eng ENG     CB  28-036  89.0           0   0  0   
6610   Aaron Cresswell  3.0  eng ENG     WB  28-046  90.0           0   0  0   
7003   Aaron Cresswell  3.0  eng ENG  WB,LB  28-050  90.0           0   0  0   
7266   Aaron Cresswell  3.0  eng ENG     CB  28-057  90.0           0   1  0   
7531   Aaron Cresswell  3.0  eng ENG     CB  28-071  90.0           0   0  0   
7770   Aaron Cresswell  3.0  eng ENG     CB  28-078  90.0           0   1  0   
8004   Aaron Cresswell  3.0  eng ENG     CB  28-085  90.0           0   0  0   
8471   Aaron Cresswell  3.0  eng ENG     CB  28-106  90.0           0   0  0   
8894   Aaron Cresswell  3.0  eng ENG     CB  28-114  63.0           0   0  0   
9158   Aaron Cresswell  3.0  eng ENG  CB,LB  28-122  90.0           0   1  0   
9363   Aaron Cresswell  3.0  eng ENG  CB,LB  28-128  90.0           0   0  0   
9651   Aaron Cresswell  3.0  eng ENG     WB  28-135  90.0           1   0  0   
9807   Aaron Cresswell  3.0  eng ENG     CB  28-141  90.0           0   0  0   
10152  Aaron Cresswell  3.0  eng ENG     CB  28-146  90.0           0   0  0   
10290  Aaron Cresswell  3.0  eng ENG     CB  28-149  87.0           0   0  0   

                                                           Expected            \
      PKatt Sh SoT CrdY CrdR Touches Press  Tkl Int Blocks       xG npxG   xA   
272       0  1   0    0    0    14.0   0.0  0.0   0    1.0      0.0  0.0  0.0   
356       0  0   0    0    0    39.0  10.0  0.0   1    2.0      0.0  0.0  0.2   
686       0  2   1    0    0    67.0   5.0  0.0   0    2.0      0.1  0.1  0.0   
1086      0  1   0    0    0    61.0  13.0  0.0   3    2.0      0.0  0.0  0.5   
1263      0  0   0    0    0    61.0   6.0  1.0   1    2.0      0.0  0.0  0.1   
1388      0  0   0    0    0    80.0  19.0  5.0   0    1.0      0.0  0.0  0.0   
1721      0  0   0    0    0    82.0  12.0  3.0   1    2.0      0.0  0.0  0.1   
2012      0  0   0    0    0    41.0  10.0  0.0   3    0.0      0.0  0.0  0.0   
2559      0  0   0    0    0    37.0  11.0  3.0   1    1.0      0.0  0.0  0.3   
2902      0  0   0    0    0    65.0  14.0  1.0   2    1.0      0.0  0.0  0.0   
32

In [9]:
np.sort(fpl17_18.opp_team_name.unique())

array(['Arsenal', 'Bournemouth', 'Brighton', 'Burnley', 'Chelsea',
       'Crystal Palace', 'Everton', 'Huddersfield', 'Leicester',
       'Liverpool', 'Man City', 'Man Utd', 'Newcastle', 'Southampton',
       'Spurs', 'Stoke', 'Swansea', 'Watford', 'West Brom', 'West Ham'],
      dtype=object)

In [10]:
np.sort(fbref17_18[('General','Opponent')].unique())

array(['Arsenal', 'Bournemouth', 'Brighton', 'Burnley', 'Chelsea',
       'Crystal Palace', 'Everton', 'Huddersfield', 'Leicester City',
       'Liverpool', 'Manchester City', 'Manchester Utd', 'Newcastle Utd',
       'Southampton', 'Stoke City', 'Swansea City', 'Tottenham',
       'Watford', 'West Brom', 'West Ham'], dtype=object)

In [11]:
my_dict = {'Leicester':'Leicester City', 'Man City':'Manchester City', 'Man Utd':'Manchester Utd', 
            'Newcastle':'Newcastle Utd', 'Stoke':'Stoke City', 'Swansea':'Swansea City', 
          'Spurs':'Tottenham'}

In [12]:
fpl17_18['opp_team_name'] = fpl17_18['opp_team_name'].map(my_dict).fillna(fpl17_18['opp_team_name'])

In [13]:
np.sort(fpl17_18['opp_team_name'].unique())

array(['Arsenal', 'Bournemouth', 'Brighton', 'Burnley', 'Chelsea',
       'Crystal Palace', 'Everton', 'Huddersfield', 'Leicester City',
       'Liverpool', 'Manchester City', 'Manchester Utd', 'Newcastle Utd',
       'Southampton', 'Stoke City', 'Swansea City', 'Tottenham',
       'Watford', 'West Brom', 'West Ham'], dtype=object)

In [14]:
ix=8117
name = fpl17_18.loc[ix, 'name']
opponent = fpl17_18.loc[ix, 'opp_team_name']
was_home = fpl17_18.loc[ix, 'was_home']
fbref17_18[(fbref17_18[('General', 'Player')]==name) & (fbref17_18[('General','Opponent')]==opponent) & 
           (fbref17_18[('General', 'HomeTeam')]==was_home)]

General                                 Performance               \
              Player    #   Nation Pos     Age   Min         Gls Ast PK PKatt   
272  Aaron Cresswell  3.0  eng ENG  LB  27-241  10.0           0   0  0     0   

                                                   Expected            SCA  \
    Sh SoT CrdY CrdR Touches Press  Tkl Int Blocks       xG npxG   xA  SCA   
272  1   0    0    0    14.0   0.0  0.0   0    1.0      0.0  0.0  0.0  1.0   

         Passes                  Carries      Dribbles        General  \
     GCA    Cmp   Att  Cmp% Prog Carries Prog     Succ  Att     Squad   
272  0.0    9.0  12.0  75.0  1.0    11.0  1.0      0.0  0.0  West Ham   

                             Total                             Short       \
    HomeTeam        Opponent   Cmp   Att  Cmp% TotDist PrgDist   Cmp  Att   
272    False  Manchester Utd   9.0  12.0  75.0   186.0    37.0   5.0  6.0   

          Medium            Long            General                            \
     Cmp%    Cmp  Att  Cmp%  Cmp  Att  Cmp%     Ast   xA   KP  1/3  PPA CrsPA   
272  83.3    1.0  2.0  50.0  3.0  4.0  75.0       0  0.0  0.0  1.0  0.0   0.0   

               Pass Types                                    Corner Kicks  \
    Prog   Att       Live Dead   FK   TB Press   Sw Crs   CK           In   
272  1.0  12.0       11.0  1.0  0.0  0.0   3.0  1.0   0  0.0          0.0   

              Height           Body Parts                       Outcomes       \
     Out  Str Ground  Low High       Left Right Head   TI Other      Cmp  Off   
272  0.0  0.0    8.0  3.0  1.0       11.0   0.0  0.0  1.0   0.0      9.0  0.0   

                     Tackles                              Vs Dribbles       \
     Out  Int Blocks     Tkl TklW Def 3rd Mid 3rd Att 3rd         Tkl  Att   
272  0.0  0.0    1.0     0.0    0     0.0     0.0     0.0         0.0  0.0   

              Pressures                                  Blocks            \
    Tkl% Past     Press Succ   % Def 3rd Mid 3rd Att 3rd Blocks   Sh ShSv   
272  NaN  0.0       0.0  0.0 NaN     0.0     0.0     0.0    1.0  0.0  0.0   

         General                   Touches                                  \
    Pass     Int Tkl+Int  Clr  Err Touches Def Pen Def 3rd Mid 3rd Att 3rd   
272  1.0       0     0.0  0.0  0.0    14.0     1.0     1.0     7.0     6.0   

                  Dribbles           Carries                              \
    Att Pen  Live    Succ%  #Pl Megs TotDist PrgDist  1/3  CPA  Mis  Dis   
272     1.0  13.0      NaN  0.0  0.0    39.0    16.0  0.0  0.0  0.0  0.0   

    Receiving                   Performance                                   \
         Targ   Rec   Rec% Prog       2CrdY Fls Fld Off Crs TklW PKwon PKcon   
272      10.0  10.0  100.0  0.0           0   0   0   0   0    0     0     0   

             Aerial Duels            
    OG Recov          Won Lost Won%  
272  0   2.0          0.0  0.0  NaN

In [15]:
names_fpl = np.sort(fpl17_18['name'].unique())
display(names_fpl)
names_fbref = np.sort(fbref17_18[('General','Player')].unique())
display(names_fbref)

array(['Aaron Cresswell', 'Aaron Lennon', 'Aaron Mooy', 'Aaron Ramsdale',
       'Aaron Ramsey', 'Aaron Wan-Bissaka', 'Abdoulaye Doucouré',
       'Adam Lallana', 'Ademola Lookman',
       'Adrián San Miguel del Castillo', 'Ainsley Maitland-Niles',
       'Alex Iwobi', 'Alex McCarthy', 'Alex Oxlade-Chamberlain',
       'Alexandre Lacazette', 'Alfie Mawson', 'Andre Gray',
       'Andreas Christensen', 'Andreas Pereira', 'Andrew Robertson',
       'Andros Townsend', 'Andy Carroll', 'Angelo Ogbonna',
       'Anthony Knockaert', 'Anthony Martial', 'Antonio Rüdiger',
       'Arthur Masuaku', 'Ashley Barnes', 'Ashley Fletcher',
       'Ashley Westwood', 'Ashley Young', 'Axel Tuanzebe',
       'Aymeric Laporte', 'Ayoze Pérez', 'Bamidele Alli', 'Ben Davies',
       'Ben Foster', 'Ben Johnson', 'Ben Mee', 'Ben Woodburn',
       'Benjamin Chilwell', 'Benjamin Mendy',
       'Bernardo Mota Veiga de Carvalho e Silva', 'Callum Hudson-Odoi',
       'Callum Wilson', 'Calum Chambers', 'Cameron Carter-

array(['Aaron Cresswell', 'Aaron Lennon', 'Aaron Mooy', 'Aaron Ramsey',
       'Aaron Wan-Bissaka', 'Abdelhamid Sabiri', 'Abdoulaye Doucouré',
       'Adam Lallana', 'Adam Smith', 'Ademola Lookman', 'Adrian Mariappa',
       'Adrien Silva', 'Adrián', 'Ahmed Hegazi', 'Ainsley Maitland-Niles',
       'Alberto Moreno', 'Aleksandar Dragović', 'Aleksandar Mitrović',
       'Alex Iwobi', 'Alex McCarthy', 'Alex Oxlade-Chamberlain',
       'Alex Pritchard', 'Alexander Sørloth', 'Alexandre Lacazette',
       'Alexis Sánchez', 'Alfie Mawson', 'Allan Nyom', 'Ander Herrera',
       'Andre Gray', 'Andreas Christensen', 'Andrew Robertson',
       'Andrew Surman', 'Andros Townsend', 'André Ayew', 'André Carrillo',
       'Andy Carroll', 'Andy King', 'Angelo Ogbonna', 'Anthony Knockaert',
       'Anthony Martial', 'Antonio Rüdiger', 'Antonio Valencia',
       'Arthur Masuaku', 'Ashley Barnes', 'Ashley Westwood',
       'Ashley Williams', 'Ashley Young', 'Asmir Begović',
       'Axel Tuanzebe', 'Aymeri

In [16]:
fbref17_18.columns = [col[0]+'_'+col[1] for col in fbref17_18.columns]
fbref17_18

,General_Player,General_#,General_Nation,General_Pos,General_Age,General_Min,Performance_Gls,Performance_Ast,Performance_PK,Performance_PKatt,Performance_Sh,Performance_SoT,Performance_CrdY,Performance_CrdR,Performance_Touches,Performance_Press,Performance_Tkl,Performance_Int,Performance_Blocks,Expected_xG,Expected_npxG,Expected_xA,SCA_SCA,SCA_GCA,Passes_Cmp,Passes_Att,Passes_Cmp%,Passes_Prog,Carries_Carries,Carries_Prog,Dribbles_Succ,Dribbles_Att,General_Squad,General_HomeTeam,General_Opponent,Total_Cmp,Total_Att,Total_Cmp%,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Short_Cmp%,Medium_Cmp,Medium_Att,Medium_Cmp%,Long_Cmp,Long_Att,Long_Cmp%,General_Ast,General_xA,General_KP,General_1/3,General_PPA,General_CrsPA,General_Prog,General_Att,Pass Types_Live,Pass Types_Dead,Pass Types_FK,Pass Types_TB,Pass Types_Press,Pass Types_Sw,Pass Types_Crs,Pass Types_CK,Corner Kicks_In,Corner Kicks_Out,Corner Kicks_Str,Height_Ground,Height_Low,Height_High,Body Parts_Left,Body Parts_Right,Body Parts_Head,Body Parts_TI,Body Parts_Other,Outcomes_Cmp,Outcomes_Off,Outcomes_Out,Outcomes_Int,Outcomes_Blocks,Tackles_Tkl,Tackles_TklW,Tackles_Def 3rd,Tackles_Mid 3rd,Tackles_Att 3rd,Vs Dribbles_Tkl,Vs Dribbles_Att,Vs Dribbles_Tkl%,Vs Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_%,Pressures_Def 3rd,Pressures_Mid 3rd,Pressures_Att 3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,General_Int,General_Tkl+Int,General_Clr,General_Err,Touches_Touches,Touches_Def Pen,Touches_Def 3rd,Touches_Mid 3rd,Touches_Att 3rd,Touches_Att Pen,Touches_Live,Dribbles_Succ%,Dribbles_#Pl,Dribbles_Megs,Carries_TotDist,Carries_PrgDist,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Rec%,Receiving_Prog,Performance_2CrdY,Performance_Fls,Performance_Fld,Performance_Off,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial Duels_Won,Aerial Duels_Lost,Aerial Duels_Won%
0,Alexandre Lacazette,9.0,fr FRA,"FW,LW",26-075,90.0,1,0,0,0,3,2,0,0,36.0,18.0,1.0,0,1.0,0.3,0.3,0.3,7.0,2.0,21.0,25.0,84.0,1.0,18.0,1.0,1.0,1.0,Arsenal,True,Leicester City,21.0,25.0,84.0,315.0,49.0,11.0,13.0,84.6,8.0,10.0,80.0,1.0,1.0,100.0,0,0.3,4.0,0.0,2.0,1.0,1.0,25.0,21.0,4.0,0.0,0.0,2.0,0.0,2,0.0,0.0,0.0,0.0,16.0,6.0,3.0,3.0,20.0,1.0,0.0,0.0,21.0,1.0,1.0,1.0,0.0,1.0,1,0.0,0.0,1.0,0.0,1.0,0.0,1.0,18.0,4.0,22.2,3.0,9.0,6.0,1.0,0.0,0.0,1.0,0,1.0,0.0,0.0,36.0,1.0,3.0,11.0,23.0,6.0,32.0,100.0,1.0,0.0,49.0,29.0,0.0,1.0,1.0,3.0,55.0,24.0,43.6,9.0,0,3,2,2,2,1,0,0,0,6.0,1.0,2.0,33.3
1,Danny Welbeck,23.0,eng ENG,"AM,RW",26-258,74.0,1,0,0,0,3,1,0,0,32.0,10.0,1.0,3,1.0,0.9,0.9,0.3,4.0,0.0,16.0,20.0,80.0,1.0,22.0,6.0,2.0,2.0,Arsenal,True,Leicester City,16.0,20.0,80.0,200.0,30.0,13.0,14.0,92.9,3.0,4.0,75.0,0.0,0.0,NaN,0,0.3,2.0,1.0,1.0,0.0,1.0,20.0,20.0,0.0,0.0,1.0,2.0,0.0,0,0.0,0.0,0.0,0.0,17.0,1.0,2.0,1.0,18.0,1.0,0.0,0.0,16.0,0.0,0.0,1.0,1.0,1.0,1,1.0,0.0,0.0,0.0,0.0,NaN,0.0,10.0,6.0,60.0,3.0,3.0,4.0,1.0,0.0,0.0,1.0,3,4.0,1.0,0.0,32.0,1.0,7.0,7.0,18.0,8.0,32.0,100.0,2.0,0.0,150.0,75.0,0.0,2.0,1.0,3.0,34.0,22.0,64.7,6.0,0,1,1,1,0,1,0,0,0,4.0,1.0,1.0,50.0
2,Theo Walcott,14.0,eng ENG,RW,28-148,16.0,0,0,0,0,0,0,0,0,3.0,1.0,0.0,1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,50.0,0.0,2.0,1.0,0.0,0.0,Arsenal,True,Leicester City,1.0,2.0,50.0,14.0,0.0,1.0,1.0,100.0,0.0,1.0,0.0,0.0,0.0,NaN,0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,1.0,0.0,0.0,3.0,0.0,1.0,0.0,2.0,0.0,3.0,NaN,0.0,0.0,19.0,16.0,0.0,0.0,0.0,0.0,5.0,2.0,40.0,0.0,0,0,0,1,1,0,0,0,0,0.0,0.0,0.0,NaN
3,Mesut Özil,11.0,de GER,AM,28-300,90.0,0,0,0,0,3,0,0,0,96.0,8.0,2.0,0,0.0,0.2,0.2,0.3,9.0,0.0,70.0,87.0,80.5,4.0,70.0,7.0,1.0,3.0,Arsenal,True,Leicester City,70.0,87.0,80.5,1177.0,183.0,39.0,44.0,88.6,21.0,24.0,87.5,8.0,13.0,61.5,0,0.3,5.0,6.0,3.0,0.0,4.0,87.0,84.0,3.0,2.0,0.0,9.0,4.0,3,1.0,0.0,1.0,0.0,71.0,6.0,10.0,77.0,8.0,1.0,0.0

In [17]:
fbref17_18 = fbref17_18.rename(columns={'General_Player':'name', 'General_Opponent':'opp_team_name', 'General_HomeTeam':'was_home'})
fbref17_18

,name,General_#,General_Nation,General_Pos,General_Age,General_Min,Performance_Gls,Performance_Ast,Performance_PK,Performance_PKatt,Performance_Sh,Performance_SoT,Performance_CrdY,Performance_CrdR,Performance_Touches,Performance_Press,Performance_Tkl,Performance_Int,Performance_Blocks,Expected_xG,Expected_npxG,Expected_xA,SCA_SCA,SCA_GCA,Passes_Cmp,Passes_Att,Passes_Cmp%,Passes_Prog,Carries_Carries,Carries_Prog,Dribbles_Succ,Dribbles_Att,General_Squad,was_home,opp_team_name,Total_Cmp,Total_Att,Total_Cmp%,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Short_Cmp%,Medium_Cmp,Medium_Att,Medium_Cmp%,Long_Cmp,Long_Att,Long_Cmp%,General_Ast,General_xA,General_KP,General_1/3,General_PPA,General_CrsPA,General_Prog,General_Att,Pass Types_Live,Pass Types_Dead,Pass Types_FK,Pass Types_TB,Pass Types_Press,Pass Types_Sw,Pass Types_Crs,Pass Types_CK,Corner Kicks_In,Corner Kicks_Out,Corner Kicks_Str,Height_Ground,Height_Low,Height_High,Body Parts_Left,Body Parts_Right,Body Parts_Head,Body Parts_TI,Body Parts_Other,Outcomes_Cmp,Outcomes_Off,Outcomes_Out,Outcomes_Int,Outcomes_Blocks,Tackles_Tkl,Tackles_TklW,Tackles_Def 3rd,Tackles_Mid 3rd,Tackles_Att 3rd,Vs Dribbles_Tkl,Vs Dribbles_Att,Vs Dribbles_Tkl%,Vs Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_%,Pressures_Def 3rd,Pressures_Mid 3rd,Pressures_Att 3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,General_Int,General_Tkl+Int,General_Clr,General_Err,Touches_Touches,Touches_Def Pen,Touches_Def 3rd,Touches_Mid 3rd,Touches_Att 3rd,Touches_Att Pen,Touches_Live,Dribbles_Succ%,Dribbles_#Pl,Dribbles_Megs,Carries_TotDist,Carries_PrgDist,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Rec%,Receiving_Prog,Performance_2CrdY,Performance_Fls,Performance_Fld,Performance_Off,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial Duels_Won,Aerial Duels_Lost,Aerial Duels_Won%
0,Alexandre Lacazette,9.0,fr FRA,"FW,LW",26-075,90.0,1,0,0,0,3,2,0,0,36.0,18.0,1.0,0,1.0,0.3,0.3,0.3,7.0,2.0,21.0,25.0,84.0,1.0,18.0,1.0,1.0,1.0,Arsenal,True,Leicester City,21.0,25.0,84.0,315.0,49.0,11.0,13.0,84.6,8.0,10.0,80.0,1.0,1.0,100.0,0,0.3,4.0,0.0,2.0,1.0,1.0,25.0,21.0,4.0,0.0,0.0,2.0,0.0,2,0.0,0.0,0.0,0.0,16.0,6.0,3.0,3.0,20.0,1.0,0.0,0.0,21.0,1.0,1.0,1.0,0.0,1.0,1,0.0,0.0,1.0,0.0,1.0,0.0,1.0,18.0,4.0,22.2,3.0,9.0,6.0,1.0,0.0,0.0,1.0,0,1.0,0.0,0.0,36.0,1.0,3.0,11.0,23.0,6.0,32.0,100.0,1.0,0.0,49.0,29.0,0.0,1.0,1.0,3.0,55.0,24.0,43.6,9.0,0,3,2,2,2,1,0,0,0,6.0,1.0,2.0,33.3
1,Danny Welbeck,23.0,eng ENG,"AM,RW",26-258,74.0,1,0,0,0,3,1,0,0,32.0,10.0,1.0,3,1.0,0.9,0.9,0.3,4.0,0.0,16.0,20.0,80.0,1.0,22.0,6.0,2.0,2.0,Arsenal,True,Leicester City,16.0,20.0,80.0,200.0,30.0,13.0,14.0,92.9,3.0,4.0,75.0,0.0,0.0,NaN,0,0.3,2.0,1.0,1.0,0.0,1.0,20.0,20.0,0.0,0.0,1.0,2.0,0.0,0,0.0,0.0,0.0,0.0,17.0,1.0,2.0,1.0,18.0,1.0,0.0,0.0,16.0,0.0,0.0,1.0,1.0,1.0,1,1.0,0.0,0.0,0.0,0.0,NaN,0.0,10.0,6.0,60.0,3.0,3.0,4.0,1.0,0.0,0.0,1.0,3,4.0,1.0,0.0,32.0,1.0,7.0,7.0,18.0,8.0,32.0,100.0,2.0,0.0,150.0,75.0,0.0,2.0,1.0,3.0,34.0,22.0,64.7,6.0,0,1,1,1,0,1,0,0,0,4.0,1.0,1.0,50.0
2,Theo Walcott,14.0,eng ENG,RW,28-148,16.0,0,0,0,0,0,0,0,0,3.0,1.0,0.0,1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,50.0,0.0,2.0,1.0,0.0,0.0,Arsenal,True,Leicester City,1.0,2.0,50.0,14.0,0.0,1.0,1.0,100.0,0.0,1.0,0.0,0.0,0.0,NaN,0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,1.0,0.0,0.0,3.0,0.0,1.0,0.0,2.0,0.0,3.0,NaN,0.0,0.0,19.0,16.0,0.0,0.0,0.0,0.0,5.0,2.0,40.0,0.0,0,0,0,1,1,0,0,0,0,0.0,0.0,0.0,NaN
3,Mesut Özil,11.0,de GER,AM,28-300,90.0,0,0,0,0,3,0,0,0,96.0,8.0,2.0,0,0.0,0.2,0.2,0.3,9.0,0.0,70.0,87.0,80.5,4.0,70.0,7.0,1.0,3.0,Arsenal,True,Leicester City,70.0,87.0,80.5,1177.0,183.0,39.0,44.0,88.6,21.0,24.0,87.5,8.0,13.0,61.5,0,0.3,5.0,6.0,3.0,0.0,4.0,87.0,84.0,3.0,2.0,0.0,9.0,4.0,3,1.0,0.0,1.0,0.0,71.0,6.0,10.0,77.0,8.0,1.0,0.0,1.0,70.0,2.0,2.0,4.0

In [18]:
# normalize spelling of names
fpl17_18['name'] = fpl17_18['name'].str.normalize('NFD').str.encode('ascii','ignore').str.decode('ascii')
fbref17_18['name'] = fbref17_18['name'].str.normalize('NFD').str.encode('ascii','ignore').str.decode('ascii')

In [19]:
fbref17_18[fbref17_18.duplicated(subset=['name','was_home','opp_team_name'])].shape[0]

43

In [20]:
fbref17_18[(fbref17_18.duplicated(subset=['name','was_home','opp_team_name'], keep=False)) & 
           (fbref17_18['name']=='Theo Walcott')]

,name,General_#,General_Nation,General_Pos,General_Age,General_Min,Performance_Gls,Performance_Ast,Performance_PK,Performance_PKatt,Performance_Sh,Performance_SoT,Performance_CrdY,Performance_CrdR,Performance_Touches,Performance_Press,Performance_Tkl,Performance_Int,Performance_Blocks,Expected_xG,Expected_npxG,Expected_xA,SCA_SCA,SCA_GCA,Passes_Cmp,Passes_Att,Passes_Cmp%,Passes_Prog,Carries_Carries,Carries_Prog,Dribbles_Succ,Dribbles_Att,General_Squad,was_home,opp_team_name,Total_Cmp,Total_Att,Total_Cmp%,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Short_Cmp%,Medium_Cmp,Medium_Att,Medium_Cmp%,Long_Cmp,Long_Att,Long_Cmp%,General_Ast,General_xA,General_KP,General_1/3,General_PPA,General_CrsPA,General_Prog,General_Att,Pass Types_Live,Pass Types_Dead,Pass Types_FK,Pass Types_TB,Pass Types_Press,Pass Types_Sw,Pass Types_Crs,Pass Types_CK,Corner Kicks_In,Corner Kicks_Out,Corner Kicks_Str,Height_Ground,Height_Low,Height_High,Body Parts_Left,Body Parts_Right,Body Parts_Head,Body Parts_TI,Body Parts_Other,Outcomes_Cmp,Outcomes_Off,Outcomes_Out,Outcomes_Int,Outcomes_Blocks,Tackles_Tkl,Tackles_TklW,Tackles_Def 3rd,Tackles_Mid 3rd,Tackles_Att 3rd,Vs Dribbles_Tkl,Vs Dribbles_Att,Vs Dribbles_Tkl%,Vs Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_%,Pressures_Def 3rd,Pressures_Mid 3rd,Pressures_Att 3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,General_Int,General_Tkl+Int,General_Clr,General_Err,Touches_Touches,Touches_Def Pen,Touches_Def 3rd,Touches_Mid 3rd,Touches_Att 3rd,Touches_Att Pen,Touches_Live,Dribbles_Succ%,Dribbles_#Pl,Dribbles_Megs,Carries_TotDist,Carries_PrgDist,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Rec%,Receiving_Prog,Performance_2CrdY,Performance_Fls,Performance_Fld,Performance_Off,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial Duels_Won,Aerial Duels_Lost,Aerial Duels_Won%
2,Theo Walcott,14.0,eng ENG,RW,28-148,16.0,0,0,0,0,0,0,0,0,3.0,1.0,0.0,1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,50.0,0.0,2.0,1.0,0.0,0.0,Arsenal,True,Leicester City,1.0,2.0,50.0,14.0,0.0,1.0,1.0,100.0,0.0,1.0,0.0,0.0,0.0,NaN,0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,1.0,0.0,0.0,3.0,0.0,1.0,0.0,2.0,0.0,3.0,NaN,0.0,0.0,19.0,16.0,0.0,0.0,0.0,0.0,5.0,2.0,40.0,0.0,0,0,0,1,1,0,0,0,0,0.0,0.0,0.0,NaN
456,Theo Walcott,14.0,eng ENG,RM,28-156,13.0,0,0,0,0,0,0,0,0,4.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,100.0,0.0,3.0,0.0,1.0,1.0,Arsenal,False,Stoke City,2.0,2.0,100.0,25.0,0.0,2.0,2.0,100.0,0.0,0.0,NaN,0.0,0.0,NaN,0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,4.0,0.0,0.0,2.0,2.0,0.0,4.0,100.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7.0,4.0,57.1,0.0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,NaN
1855,Theo Walcott,14.0,eng ENG,AM,28-199,20.0,0,0,0,0,0,0,0,0,8.0,3.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,8.0,62.5,0.0,6.0,2.0,0.0,0.0,Arsenal,True,Brighton,5.0,8.0,62.5,66.0,4.0,4.0,7.0,57.1,1.0,1.0,100.0,0.0,0.0,NaN,0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,7.0,1.0,0.0,0.0,1.0,0.0,0,0.0,0.0,0.0,0.0,6.0,1.0,1.0,0.0,7.0,0.0,1.0,0.0,5.0,0.0,0.0,0.0,2.0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,3.0,1.0,33.3,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,8.0,0.0,1.0,3.0,5.0,1.0,7.0,NaN,0.0,0.0,33.0,25.0,1.0,0.0,0.0,0.0,12.0,7.0,58.3,2.0,0,0,0,1,0,0,0,0,0,0.0,0.0,2.0,0.0
4960,Theo Walcott,14.0,eng ENG,LW,28-281,2.0,0,0,0,0,0,0,0,0,2.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,1.0,1.0,1.0,Arsenal,True,Liverpool,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,1.0,0.0,1,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,2.0,0.

In [21]:
df = fpl17_18.merge(fbref17_18, how='inner', on=['name','was_home','opp_team_name'])
df

,season_x,name,position,team_x,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,opp_team_name,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW,General_#,General_Nation,General_Pos,General_Age,General_Min,Performance_Gls,Performance_Ast,Performance_PK,Performance_PKatt,Performance_Sh,Performance_SoT,Performance_CrdY,Performance_CrdR,Performance_Touches,Performance_Press,Performance_Tkl,Performance_Int,Performance_Blocks,Expected_xG,Expected_npxG,Expected_xA,SCA_SCA,SCA_GCA,Passes_Cmp,Passes_Att,Passes_Cmp%,Passes_Prog,Carries_Carries,Carries_Prog,Dribbles_Succ,Dribbles_Att,General_Squad,Total_Cmp,Total_Att,Total_Cmp%,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Short_Cmp%,Medium_Cmp,Medium_Att,Medium_Cmp%,Long_Cmp,Long_Att,Long_Cmp%,General_Ast,General_xA,General_KP,General_1/3,General_PPA,General_CrsPA,General_Prog,General_Att,Pass Types_Live,Pass Types_Dead,Pass Types_FK,Pass Types_TB,Pass Types_Press,Pass Types_Sw,Pass Types_Crs,Pass Types_CK,Corner Kicks_In,Corner Kicks_Out,Corner Kicks_Str,Height_Ground,Height_Low,Height_High,Body Parts_Left,Body Parts_Right,Body Parts_Head,Body Parts_TI,Body Parts_Other,Outcomes_Cmp,Outcomes_Off,Outcomes_Out,Outcomes_Int,Outcomes_Blocks,Tackles_Tkl,Tackles_TklW,Tackles_Def 3rd,Tackles_Mid 3rd,Tackles_Att 3rd,Vs Dribbles_Tkl,Vs Dribbles_Att,Vs Dribbles_Tkl%,Vs Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_%,Pressures_Def 3rd,Pressures_Mid 3rd,Pressures_Att 3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,General_Int,General_Tkl+Int,General_Clr,General_Err,Touches_Touches,Touches_Def Pen,Touches_Def 3rd,Touches_Mid 3rd,Touches_Att 3rd,Touches_Att Pen,Touches_Live,Dribbles_Succ%,Dribbles_#Pl,Dribbles_Megs,Carries_TotDist,Carries_PrgDist,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Rec%,Receiving_Prog,Performance_2CrdY,Performance_Fls,Performance_Fld,Performance_Off,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial Duels_Won,Aerial Duels_Lost,Aerial Duels_Won%
0,2017-18,Aaron Cresswell,DEF,NaN,0,0,3,0,0.6,443,9,2,0,1.9,0.4,2017-08-13 15:00:00+00:00,9,12,Manchester Utd,0,0,0,0,1,0,25136,0.0,4.0,18.0,0,0,0,0,50,False,0,1,3.0,eng ENG,LB,27-241,10.0,0,0,0,0,1,0,0,0,14.0,0.0,0.0,0,1.0,0.0,0.0,0.0,1.0,0.0,9.0,12.0,75.0,1.0,11.0,1.0,0.0,0.0,West Ham,9.0,12.0,75.0,186.0,37.0,5.0,6.0,83.3,1.0,2.0,50.0,3.0,4.0,75.0,0,0.0,0.0,1.0,0.0,0.0,1.0,12.0,11.0,1.0,0.0,0.0,3.0,1.0,0,0.0,0.0,0.0,0.0,8.0,3.0,1.0,11.0,0.0,0.0,1.0,0.0,9.0,0.0,0.0,0.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0,0.0,0.0,0.0,14.0,1.0,1.0,7.0,6.0,1.0,13.0,NaN,0.0,0.0,39.0,16.0,0.0,0.0,0.0,0.0,10.0,10.0,100.0,0.0,0,0,0,0,0,0,0,0,0,2.0,0.0,0.0,NaN
1,2017-18,Aaron Mooy,MID,NaN,1,0,22,1,46.9,172,4,0,0,8.7,40.2,2017-08-12 14:00:00+00:00,90,6,Crystal Palace,0,0,0,0,1,0,59955,3.0,0.0,0.0,6,0,0,0,55,False,0,1,10.0,au AUS,DM,26-331,90.0,0,1,0,0,0,0,0,0,74.0,15.0,2.0,2,2.0,0.0,0.0,0.3,4.0,1.0,49.0,65.0,75.4,2.0,42.0,4.0,0.0,0.0,Huddersfield,49.0,65.0,75.4,839.0,336.0,25.0,26.0,96.2,13.0,16.0,81.3,8.0,16.0,50.0,1,0.3,3.0,5.0,1.0,1.0,2.0,65.0,57.0,8.0,1.0,0.0,13.0,3.0,1,7.0,5.0,0.0,0.0,39.0,6.0,20.0,21.0,41.0,3.0,0.0,0.0,49.0,0.0,1.0,0.0,6.0,2.0,2,1.0,1.0,0.0,2.0,3.0,66.7,1.0,15.0,9.0,60.0,4.0,8.0,3.0,2.0,1.0,0.0,1.0,2,4.0,2.0,0.0,74.0,4.0,18.0,34.0,28.0,0.0,66.0,NaN,0.0,0.0,216.0,80.0,2.0,0.0,1.0,1.0,47.0,44.0,93.6,0.0,0,0,0,0,1,2,0,0,0,12.0,0.0,0.0,NaN
2,2017-18,Aaron Ramsey,MID,NaN,0,0,16,0,11.2,18,1,0,1,6.7,29.6,2017-08-11 18:45:00+00:00,23,9,Leicester City,0,0,0,0,1,0,33792,3.0,4.0,26.0,6,0,0,0,70,True,0,1,8.0,wls WAL,DM,26-228,24.0,1,0,0,0,4,1,0,0,19.0,2.0,0.0,0,1.0,0.4,0.4,0.0,1.0,0.0,8.0,11.0,72.7,1.0,13.0,1.0,0.0,0.0,Arsenal,8.0,11.0,72.7,149.0,61.0,3.0,5.0

In [22]:
df['total_points'].isnull().sum()

0

In [ ]:
#df_left = fpl17_18.merge(fbref17_18, how='left', on=['name','was_home','opp_team_name'])
#df_left

In [ ]:
#df_left['total_points'].isnull().sum()

In [ ]:
#df_left[df_left['General_#'].isnull()]

In [ ]:
df[df['name']=='Aaron Lennon'].sort_values(by='kickoff_time')

In [ ]:
#df_left[df_left['name']=='Aaron Lennon']

In [ ]:
fbref17_18[fbref17_18['name']=='Aaron Lennon']

In [ ]:
fpl17_18[fpl17_18['name']=='Aaron Lennon'].sort_values(by='kickoff_time')

In [ ]:
print(fpl17_18[fpl17_18['name']=='Aaron Lennon'].shape[0])
print(fbref17_18[fbref17_18['name']=='Aaron Lennon'].shape[0])
print(df[df['name']=='Aaron Lennon'].shape[0])
#print(df_left[df_left['name']=='Aaron Lennon'].shape[0])

In [ ]:
fpl17_18[fpl17_18['minutes']>0].shape[0]

In [ ]:
fpl17_18[fpl17_18['minutes']==0].shape[0]

In [ ]:
fpl17_18[fpl17_18['minutes'].isnull()].shape[0]

In [ ]:
22*380

In [ ]:
df[(df['name']=='Theo Walcott') & (df['was_home']==True) & (df['opp_team_name']=='Leicester City')]

In [ ]:
df[(df['name']=='Aaron Cresswell')]

In [23]:
all_names_list = df['name'].unique()
np.sort(all_names_list)

array(['Aaron Cresswell', 'Aaron Lennon', 'Aaron Mooy', 'Aaron Ramsey',
       'Aaron Wan-Bissaka', 'Abdoulaye Doucoure', 'Adam Lallana',
       'Ademola Lookman', 'Ainsley Maitland-Niles', 'Alex Iwobi',
       'Alex McCarthy', 'Alex Oxlade-Chamberlain', 'Alexandre Lacazette',
       'Alfie Mawson', 'Andre Gray', 'Andreas Christensen',
       'Andrew Robertson', 'Andros Townsend', 'Andy Carroll',
       'Angelo Ogbonna', 'Anthony Knockaert', 'Anthony Martial',
       'Antonio Rudiger', 'Arthur Masuaku', 'Ashley Barnes',
       'Ashley Westwood', 'Ashley Young', 'Axel Tuanzebe',
       'Aymeric Laporte', 'Ayoze Perez', 'Ben Davies', 'Ben Foster',
       'Ben Mee', 'Ben Woodburn', 'Benjamin Mendy', 'Callum Hudson-Odoi',
       'Callum Wilson', 'Calum Chambers', 'Cedric Soares', 'Cenk Tosun',
       'Cesar Azpilicueta', 'Charlie Austin', 'Charlie Taylor',
       'Cheikhou Kouyate', 'Chris Smalling', 'Chris Wood',
       'Christian Atsu', 'Christian Benteke', 'Christian Eriksen',
       'C

In [ ]:
print(len(all_names_list))
print(fpl17_18['name'].nunique())
print(fbref17_18['name'].nunique())

In [ ]:
# these names are missing from df
set(fpl17_18['name'].unique()).difference(set(all_names_list))

In [ ]:
aux = df[(df['name']=='Aaron Cresswell')].sort_values(by='kickoff_time').copy()
aux

In [24]:
# change player values to correct units
df['value'] = df['value']/10.0

Calculate xPoints

In [26]:
one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False).\
                        fit(np.array(df['position']).reshape(-1,1))
one_hot_encoder.get_feature_names_out()

array(['x0_DEF', 'x0_FWD', 'x0_GK', 'x0_MID'], dtype=object)

In [27]:
one_hot_encoder.transform(np.array(df['position']).reshape(-1,1))

array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.]])

In [28]:
fpl_positions = ['fpl_position_'+col[3:] for col in one_hot_encoder.get_feature_names_out()]
fpl_positions

['fpl_position_DEF', 'fpl_position_FWD', 'fpl_position_GK', 'fpl_position_MID']

In [29]:
df[fpl_positions] = one_hot_encoder.transform(np.array(df['position']).reshape(-1,1))
#df = df.drop(columns=['position'])
df

,season_x,name,team_x,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,opp_team_name,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW,General_#,General_Nation,General_Pos,General_Age,General_Min,Performance_Gls,Performance_Ast,Performance_PK,Performance_PKatt,Performance_Sh,Performance_SoT,Performance_CrdY,Performance_CrdR,Performance_Touches,Performance_Press,Performance_Tkl,Performance_Int,Performance_Blocks,Expected_xG,Expected_npxG,Expected_xA,SCA_SCA,SCA_GCA,Passes_Cmp,Passes_Att,Passes_Cmp%,Passes_Prog,Carries_Carries,Carries_Prog,Dribbles_Succ,Dribbles_Att,General_Squad,Total_Cmp,Total_Att,Total_Cmp%,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Short_Cmp%,Medium_Cmp,Medium_Att,Medium_Cmp%,Long_Cmp,Long_Att,Long_Cmp%,General_Ast,General_xA,General_KP,General_1/3,General_PPA,General_CrsPA,General_Prog,General_Att,Pass Types_Live,Pass Types_Dead,Pass Types_FK,Pass Types_TB,Pass Types_Press,Pass Types_Sw,Pass Types_Crs,Pass Types_CK,Corner Kicks_In,Corner Kicks_Out,Corner Kicks_Str,Height_Ground,Height_Low,Height_High,Body Parts_Left,Body Parts_Right,Body Parts_Head,Body Parts_TI,Body Parts_Other,Outcomes_Cmp,Outcomes_Off,Outcomes_Out,Outcomes_Int,Outcomes_Blocks,Tackles_Tkl,Tackles_TklW,Tackles_Def 3rd,Tackles_Mid 3rd,Tackles_Att 3rd,Vs Dribbles_Tkl,Vs Dribbles_Att,Vs Dribbles_Tkl%,Vs Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_%,Pressures_Def 3rd,Pressures_Mid 3rd,Pressures_Att 3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,General_Int,General_Tkl+Int,General_Clr,General_Err,Touches_Touches,Touches_Def Pen,Touches_Def 3rd,Touches_Mid 3rd,Touches_Att 3rd,Touches_Att Pen,Touches_Live,Dribbles_Succ%,Dribbles_#Pl,Dribbles_Megs,Carries_TotDist,Carries_PrgDist,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Rec%,Receiving_Prog,Performance_2CrdY,Performance_Fls,Performance_Fld,Performance_Off,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial Duels_Won,Aerial Duels_Lost,Aerial Duels_Won%,fpl_position_DEF,fpl_position_FWD,fpl_position_GK,fpl_position_MID
0,2017-18,Aaron Cresswell,NaN,0,0,3,0,0.6,443,9,2,0,1.9,0.4,2017-08-13 15:00:00+00:00,9,12,Manchester Utd,0,0,0,0,1,0,25136,0.0,4.0,18.0,0,0,0,0,5.0,False,0,1,3.0,eng ENG,LB,27-241,10.0,0,0,0,0,1,0,0,0,14.0,0.0,0.0,0,1.0,0.0,0.0,0.0,1.0,0.0,9.0,12.0,75.0,1.0,11.0,1.0,0.0,0.0,West Ham,9.0,12.0,75.0,186.0,37.0,5.0,6.0,83.3,1.0,2.0,50.0,3.0,4.0,75.0,0,0.0,0.0,1.0,0.0,0.0,1.0,12.0,11.0,1.0,0.0,0.0,3.0,1.0,0,0.0,0.0,0.0,0.0,8.0,3.0,1.0,11.0,0.0,0.0,1.0,0.0,9.0,0.0,0.0,0.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0,0.0,0.0,0.0,14.0,1.0,1.0,7.0,6.0,1.0,13.0,NaN,0.0,0.0,39.0,16.0,0.0,0.0,0.0,0.0,10.0,10.0,100.0,0.0,0,0,0,0,0,0,0,0,0,2.0,0.0,0.0,NaN,1.0,0.0,0.0,0.0
1,2017-18,Aaron Mooy,NaN,1,0,22,1,46.9,172,4,0,0,8.7,40.2,2017-08-12 14:00:00+00:00,90,6,Crystal Palace,0,0,0,0,1,0,59955,3.0,0.0,0.0,6,0,0,0,5.5,False,0,1,10.0,au AUS,DM,26-331,90.0,0,1,0,0,0,0,0,0,74.0,15.0,2.0,2,2.0,0.0,0.0,0.3,4.0,1.0,49.0,65.0,75.4,2.0,42.0,4.0,0.0,0.0,Huddersfield,49.0,65.0,75.4,839.0,336.0,25.0,26.0,96.2,13.0,16.0,81.3,8.0,16.0,50.0,1,0.3,3.0,5.0,1.0,1.0,2.0,65.0,57.0,8.0,1.0,0.0,13.0,3.0,1,7.0,5.0,0.0,0.0,39.0,6.0,20.0,21.0,41.0,3.0,0.0,0.0,49.0,0.0,1.0,0.0,6.0,2.0,2,1.0,1.0,0.0,2.0,3.0,66.7,1.0,15.0,9.0,60.0,4.0,8.0,3.0,2.0,1.0,0.0,1.0,2,4.0,2.0,0.0,74.0,4.0,18.0,34.0,28.0,0.0,66.0,NaN,0.0,0.0,216.0,80.0,2.0,0.0,1.0,1.0,47.0,44.0,93.6,0.0,0,0,0,0,1,2,0,0,0,12.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0
2,2017-18,Aaron Ramsey,NaN,0,0,16,0,11.2,18,1,0,1,6.7,29.6,2017-08-11 18:45:00+00:00,23,9,Leicester City,0,0,0,0,1,0,33792,3.0,4.0,26.0,6,0,0,0,7.0,True,0,1,8.0,wls WAL,DM,26-228,24.0,1,0,0,0,4,1,0,0,19.0,2.0,0.0,0,1.0,0.4,0.4,0.0,1.

In [30]:
fpl_positions_ordered = [ 'fpl_position_GK', 'fpl_position_DEF', 'fpl_position_MID', 'fpl_position_FWD']

In [31]:
aux = df[fpl_positions_ordered].copy()
df = df.drop(columns=fpl_positions)
df = pd.concat([df, aux], axis=1)
df

,season_x,name,team_x,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,opp_team_name,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW,General_#,General_Nation,General_Pos,General_Age,General_Min,Performance_Gls,Performance_Ast,Performance_PK,Performance_PKatt,Performance_Sh,Performance_SoT,Performance_CrdY,Performance_CrdR,Performance_Touches,Performance_Press,Performance_Tkl,Performance_Int,Performance_Blocks,Expected_xG,Expected_npxG,Expected_xA,SCA_SCA,SCA_GCA,Passes_Cmp,Passes_Att,Passes_Cmp%,Passes_Prog,Carries_Carries,Carries_Prog,Dribbles_Succ,Dribbles_Att,General_Squad,Total_Cmp,Total_Att,Total_Cmp%,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Short_Cmp%,Medium_Cmp,Medium_Att,Medium_Cmp%,Long_Cmp,Long_Att,Long_Cmp%,General_Ast,General_xA,General_KP,General_1/3,General_PPA,General_CrsPA,General_Prog,General_Att,Pass Types_Live,Pass Types_Dead,Pass Types_FK,Pass Types_TB,Pass Types_Press,Pass Types_Sw,Pass Types_Crs,Pass Types_CK,Corner Kicks_In,Corner Kicks_Out,Corner Kicks_Str,Height_Ground,Height_Low,Height_High,Body Parts_Left,Body Parts_Right,Body Parts_Head,Body Parts_TI,Body Parts_Other,Outcomes_Cmp,Outcomes_Off,Outcomes_Out,Outcomes_Int,Outcomes_Blocks,Tackles_Tkl,Tackles_TklW,Tackles_Def 3rd,Tackles_Mid 3rd,Tackles_Att 3rd,Vs Dribbles_Tkl,Vs Dribbles_Att,Vs Dribbles_Tkl%,Vs Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_%,Pressures_Def 3rd,Pressures_Mid 3rd,Pressures_Att 3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,General_Int,General_Tkl+Int,General_Clr,General_Err,Touches_Touches,Touches_Def Pen,Touches_Def 3rd,Touches_Mid 3rd,Touches_Att 3rd,Touches_Att Pen,Touches_Live,Dribbles_Succ%,Dribbles_#Pl,Dribbles_Megs,Carries_TotDist,Carries_PrgDist,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Rec%,Receiving_Prog,Performance_2CrdY,Performance_Fls,Performance_Fld,Performance_Off,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial Duels_Won,Aerial Duels_Lost,Aerial Duels_Won%,fpl_position_GK,fpl_position_DEF,fpl_position_MID,fpl_position_FWD
0,2017-18,Aaron Cresswell,NaN,0,0,3,0,0.6,443,9,2,0,1.9,0.4,2017-08-13 15:00:00+00:00,9,12,Manchester Utd,0,0,0,0,1,0,25136,0.0,4.0,18.0,0,0,0,0,5.0,False,0,1,3.0,eng ENG,LB,27-241,10.0,0,0,0,0,1,0,0,0,14.0,0.0,0.0,0,1.0,0.0,0.0,0.0,1.0,0.0,9.0,12.0,75.0,1.0,11.0,1.0,0.0,0.0,West Ham,9.0,12.0,75.0,186.0,37.0,5.0,6.0,83.3,1.0,2.0,50.0,3.0,4.0,75.0,0,0.0,0.0,1.0,0.0,0.0,1.0,12.0,11.0,1.0,0.0,0.0,3.0,1.0,0,0.0,0.0,0.0,0.0,8.0,3.0,1.0,11.0,0.0,0.0,1.0,0.0,9.0,0.0,0.0,0.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0,0.0,0.0,0.0,14.0,1.0,1.0,7.0,6.0,1.0,13.0,NaN,0.0,0.0,39.0,16.0,0.0,0.0,0.0,0.0,10.0,10.0,100.0,0.0,0,0,0,0,0,0,0,0,0,2.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0
1,2017-18,Aaron Mooy,NaN,1,0,22,1,46.9,172,4,0,0,8.7,40.2,2017-08-12 14:00:00+00:00,90,6,Crystal Palace,0,0,0,0,1,0,59955,3.0,0.0,0.0,6,0,0,0,5.5,False,0,1,10.0,au AUS,DM,26-331,90.0,0,1,0,0,0,0,0,0,74.0,15.0,2.0,2,2.0,0.0,0.0,0.3,4.0,1.0,49.0,65.0,75.4,2.0,42.0,4.0,0.0,0.0,Huddersfield,49.0,65.0,75.4,839.0,336.0,25.0,26.0,96.2,13.0,16.0,81.3,8.0,16.0,50.0,1,0.3,3.0,5.0,1.0,1.0,2.0,65.0,57.0,8.0,1.0,0.0,13.0,3.0,1,7.0,5.0,0.0,0.0,39.0,6.0,20.0,21.0,41.0,3.0,0.0,0.0,49.0,0.0,1.0,0.0,6.0,2.0,2,1.0,1.0,0.0,2.0,3.0,66.7,1.0,15.0,9.0,60.0,4.0,8.0,3.0,2.0,1.0,0.0,1.0,2,4.0,2.0,0.0,74.0,4.0,18.0,34.0,28.0,0.0,66.0,NaN,0.0,0.0,216.0,80.0,2.0,0.0,1.0,1.0,47.0,44.0,93.6,0.0,0,0,0,0,1,2,0,0,0,12.0,0.0,0.0,NaN,0.0,0.0,1.0,0.0
2,2017-18,Aaron Ramsey,NaN,0,0,16,0,11.2,18,1,0,1,6.7,29.6,2017-08-11 18:45:00+00:00,23,9,Leicester City,0,0,0,0,1,0,33792,3.0,4.0,26.0,6,0,0,0,7.0,True,0,1,8.0,wls WAL,DM,26-228,24.0,1,0,0,0,4,1,0,0,19.0,2.0,0.0,0,1.0,0.4,0.4,0.0,1.

Calculate statistics

In [ ]:
# name will be dropped later but is needed until we have used fbref position info
general_columns = ['name','GW', 'General_Squad', 'opp_team_name', 'kickoff_time']

columns_for_difference = ['selected', 'transfers_balance', 'value']

columns_for_ewa = ['total_points', 'assists', 'bonus', 'bps', 'clean_sheets', 'creativity', 
                   'goals_conceded', 'goals_scored', 'ict_index', 'influence', 'minutes', 'own_goals', 'penalties_missed',
                   'penalties_saved', 'saves', 'threat', 'yellow_cards', 
                'Performance_Sh', 'Performance_SoT', 'Performance_Touches', 'Performance_Press', 'Performance_Tkl',
                  'Performance_Int', 'Performance_Blocks', 'Expected_xG', 'Expected_npxG', 'Expected_xA',
                  'SCA_SCA', 'Passes_Cmp', 'Passes_Cmp%', 'Passes_Prog', 'Carries_Carries', 'Carries_Prog',
                  'Dribbles_Succ', 'Dribbles_Att', 'General_KP', 'Touches_Att 3rd', 'Touches_Att Pen', 
                   'Performance_Recov', 'Aerial Duels_Won', 'Aerial Duels_Won%']

categorical_columns = ['position','General_Pos', 'was_home']

player_names = df['name'].unique()
all_data = pd.DataFrame()

for player_name in player_names:
    # data for modelling for given player
    data = pd.DataFrame()
    # all data for given player
    aux = df[(df['name']==player_name)].sort_values(by='kickoff_time').copy()
    
    # copy general player data 
    data[general_columns] = aux[general_columns].copy()
    # add player data for calculating various differences over gameweeks
    data[columns_for_difference] = aux[columns_for_difference].copy()
    # add player data for exponentially weighted averages
    data[columns_for_ewa] = aux[columns_for_ewa].copy()
    # add categorical information
    data[categorical_columns] = aux[categorical_columns].copy()
    
    # info on player availability
    for i in [1,5,10]:#range(1,11):
        data[f'number_missed games_in_{i}'] = (data['minutes']==0).ewm(alpha=1/i).mean().shift(1)

    # change in fpl ownership and value
    # shift values down one row (info known from previous game weeks before a new game)
    data[columns_for_difference] = data[columns_for_difference].shift(1)
    # change in fpl ownership 
    data['transfers_balance_%'] = data['transfers_balance'] / data['selected']
    data['transfers_balance_%_ewa5'] = data['transfers_balance_%'].ewm(alpha=1/5).mean()
    data['transfers_balance_%_ewa10'] = data['transfers_balance_%'].ewm(alpha=1/10).mean()
    # change in fpl value
    for i in [1,2,5]:
        data[f'value_change_{i}'] = data['value'].diff(i)
    
    # drop rows for games where the player did not play so these are not included when counting exponentially 
    # weighted averages
    data = data[data['minutes']>0]
    # calculate exponential weighted average for a number of statistics and shift down 1 row 
    # (so that this is the info known when trying to predict total_points of the row)
    for i in [1,5,10]:#range(1,11):
        new_data = data[columns_for_ewa].ewm(alpha=1/i).mean().shift(1)
        new_columns = [col+f'_ewa_{i}' for col in columns_for_ewa]
        new_data.columns = new_columns
        data = pd.concat([data, new_data], axis=1)

    # drop non-model-input data (leave 'total_points' i.e. modelling target variable)
    data = data.drop(columns=columns_for_ewa[1:])
    data = data.drop(columns=['selected', 'transfers_balance'])
    # add given player's data to the dataframe for all players
    all_data = pd.concat([all_data,data], ignore_index=True)

display(all_data)

In [ ]:
all_data['position'].unique()

In [ ]:
one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False).\
                        fit(np.array(all_data['position']).reshape(-1,1))
one_hot_encoder.get_feature_names_out()

In [ ]:
one_hot_encoder.transform(np.array(all_data['position']).reshape(-1,1))

In [ ]:
fpl_positions = ['fpl_position_'+col[3:] for col in one_hot_encoder.get_feature_names_out()]
fpl_positions

In [ ]:
all_data[fpl_positions] = one_hot_encoder.transform(np.array(all_data['position']).reshape(-1,1))
all_data = all_data.drop(columns=['position'])
all_data

In [ ]:
fpl_positions_ordered = [ 'fpl_position_GK', 'fpl_position_DEF', 'fpl_position_MID', 'fpl_position_FWD']

In [ ]:
aux = all_data[fpl_positions_ordered].copy()
all_data = all_data.drop(columns=fpl_positions)
all_data = pd.concat([all_data, aux], axis=1)
all_data

In [ ]:
all_data['was_home'] = all_data['was_home'].apply(lambda x: 1 if x else 0)

In [ ]:
#one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False).\
#                        fit(np.array(all_data['General_Pos']).reshape(-1,1))
#one_hot_encoder.get_feature_names_out()

In [ ]:
#new_columns = ['fbref_pos'+col[2:] for col in one_hot_encoder.get_feature_names_out()]
#new_columns

In [ ]:
all_data['General_Pos'].str.split(',')

In [ ]:
mlb = MultiLabelBinarizer()

In [ ]:
fbref_position_encoding = mlb.fit_transform(all_data.loc[all_data['General_Pos'].notnull(), 'General_Pos'].str.split(','))
fbref_position_encoding

In [ ]:
mlb.classes_

In [ ]:
fbref_position_columns = ['fbref_position_'+position for position in mlb.classes_]
np.array(fbref_position_columns)

In [ ]:
all_data.loc[all_data['General_Pos'].notnull(), fbref_position_columns] = fbref_position_encoding
all_data = all_data.drop(columns=['General_Pos'])
all_data

In [ ]:
fbref_position_columns_ordered = ['fbref_position_GK', 'fbref_position_LB', 'fbref_position_CB', 'fbref_position_RB', 
                'fbref_position_WB', 'fbref_position_DM', 'fbref_position_LM', 'fbref_position_CM', 
                'fbref_position_RM', 'fbref_position_AM', 'fbref_position_LW', 'fbref_position_RW',
                'fbref_position_FW', 
                ]

In [ ]:
aux = all_data[fbref_position_columns_ordered].copy()
all_data = all_data.drop(columns=fbref_position_columns)
all_data = pd.concat([all_data, aux], axis=1)
all_data

In [ ]:
#player_name = player_names[0]
#aux = all_data[(all_data['name']==player_name)].copy()
#for i in [5,10]:
#    new_columns = [col+f'_ewa{i}' for col in fbref_position_columns]
#    aux[new_columns] = aux[fbref_position_columns].ewm(alpha=1/i).mean().shift(1)
#aux[new_columns]

In [ ]:
# calculate weighted averages of fbref positions for each player, add to all_data, and remove original fbref 
# position data (not known for prediction purposes)
new_column_data = pd.DataFrame()
new_columns_5 = [col+'_ewa5' for col in fbref_position_columns_ordered]
new_columns_10 = [col+'_ewa10' for col in fbref_position_columns_ordered]
for player_name in player_names:
    aux = all_data[(all_data['name']==player_name)].copy()
    aux[new_columns_5] = aux[fbref_position_columns_ordered].ewm(alpha=1/5).mean().shift(1)
    aux[new_columns_10] = aux[fbref_position_columns_ordered].ewm(alpha=1/10).mean().shift(1)
    aux = pd.concat([aux[new_columns_5],aux[new_columns_10]], axis=1)
    new_column_data = pd.concat([new_column_data, aux], axis=0)

# add data to all_data
all_data = pd.concat([all_data,new_column_data], axis=1)
all_data = all_data.drop(columns=fbref_position_columns_ordered)

display(all_data)

In [ ]:
# we can now remove player names from all_data
all_data = all_data.drop(columns=['name'])
all_data

## Create a lookup table for team xG and xGA weighted rolling averages over the season

In [ ]:
fixtures = pd.read_csv('/Users/jaakkotoivonen/Documents/my_stuff/football-data-analysis/data/match-results/premier-league/season17-18.csv')
fixtures = fixtures.rename(columns={'xG':'xG_home', 'xG.1':'xG_away'})
fixtures['Date'] = pd.to_datetime(fixtures['Date'], yearfirst=True, utc=True)
fixtures

In [ ]:
# remove null rows
fixtures = fixtures[fixtures.isnull().sum(axis=1)<14]
fixtures

In [ ]:
display(np.sort(all_data['General_Squad'].unique()))
display(np.sort(fixtures.Home.unique()))

In [ ]:
team_xG_data = []
for ix in fixtures.index:
    date = fixtures.loc[ix,'Date']
    home_team = fixtures.loc[ix,'Home']
    away_team = fixtures.loc[ix,'Away']
    xG_home = fixtures.loc[ix,'xG_home']
    xG_away = fixtures.loc[ix,'xG_away']
    team_xG_data.append([date, home_team, xG_home, xG_away])
    team_xG_data.append([date, away_team, xG_away, xG_home])

team_xG = pd.DataFrame(data=team_xG_data, columns=['date','team','xG','xGA'])
display(team_xG)

In [ ]:
team_xG_ewa = pd.DataFrame(columns=['date','team','team_xG_ewa_1','team_xG_ewa_5','team_xG_ewa_10',
                                   'team_xGA_ewa_1','team_xGA_ewa_5','team_xGA_ewa_10'])
team_xG_ewa

In [ ]:
all_team_names = fixtures.Home.unique()
for team in all_team_names:
    aux = team_xG[team_xG['team']==team].copy()
    for i in [1,5,10]:
        aux[f'team_xG_ewa_{i}'] = aux['xG'].ewm(alpha=1/i).mean()
        aux[f'team_xGA_ewa_{i}'] = aux['xGA'].ewm(alpha=1/i).mean()
    team_xG_ewa = pd.concat([team_xG_ewa, aux[['date','team','team_xG_ewa_1','team_xG_ewa_5','team_xG_ewa_10',
                                   'team_xGA_ewa_1','team_xGA_ewa_5','team_xGA_ewa_10']]])
display(team_xG_ewa)

Let's use the lookup table to add team strength info to player data

In [ ]:
aux_df_all = pd.DataFrame()
for ix in all_data.index:
    my_team = all_data.loc[ix, 'General_Squad']
    opponent_team = all_data.loc[ix, 'opp_team_name']
    game_date = all_data.loc[ix, 'kickoff_time'] 
    # "game_date - one day" needed because team_xG_ewa date is marked on midnight and all_data uses actual game starting time
    aux = team_xG_ewa[(team_xG_ewa['date']<(game_date - dt.timedelta(days=1))) & (team_xG_ewa['team']==my_team)].tail(1)\
                        [['team_xG_ewa_1','team_xG_ewa_5','team_xG_ewa_10','team_xGA_ewa_1','team_xGA_ewa_5',
                          'team_xGA_ewa_10']].copy()
    if aux.shape[0]==1:
        aux.index = [ix]
    aux2 = team_xG_ewa[(team_xG_ewa['date']<(game_date - dt.timedelta(days=1))) & 
                       (team_xG_ewa['team']==opponent_team)].tail(1)\
                        [['team_xG_ewa_1','team_xG_ewa_5','team_xG_ewa_10','team_xGA_ewa_1','team_xGA_ewa_5',
                          'team_xGA_ewa_10']].copy()
    aux2.columns = ['opp_'+col for col in aux.columns]
    if aux2.shape[0]==1:
        aux2.index = [ix]
    aux_final = pd.concat([aux,aux2], axis=1)
    aux_df_all = pd.concat([aux_df_all, aux_final])

display(aux_df_all)

In [ ]:
all_data = pd.concat([all_data, aux_df_all], axis=1)
display(all_data)

In [ ]:
fpl_positions_ordered

In [ ]:
all_data.loc[0,fpl_positions_ordered].values *

In [ ]:
np.array(all_data.columns)